Biostat/Biomath M257 Homework 6

Due June 7 @ 11:59PM

Tomoki Okuno and 805851067

System information (for reproducibility):

In [3]:
versioninfo()

Julia Version 1.10.0
Commit 3120989f39b (2023-12-25 18:01 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 8 × Apple M1
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, apple-m1)
  Threads: 2 on 4 virtual cores


Load packages:

In [4]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/07_UCLA/Class/257/02_Homework/hw6`


Status `~/Documents/07_UCLA/Class/257/02_Homework/hw6/Project.toml`
  [6e4b80f9] BenchmarkTools v1.5.0
  [336ed68f] CSV v0.10.14
  [a93c6f00] DataFrames v1.6.1
  [8bb1440f] DelimitedFiles v1.9.1
  [31c24e10] Distributions v0.25.108
  [b6b21f68] Ipopt v1.6.2
  [67920dd8] KNITRO v0.14.2
  [b8f27783] MathOptInterface v1.30.0
  [ff71e718] MixedModels v4.24.0
  [76087f3c] NLopt v1.0.2
  [08abe8d2] PrettyTables v2.3.2
  [6f49c342] RCall v0.14.1
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random


In this assignment, we continue with the linear mixed effects model (LMM) considered in HW3
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma}_i + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effects predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effects predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma}_i \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

The log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ is 
$$
    \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma_0^2) = - \frac{n_i}{2} \log (2\pi) - \frac{1}{2} \log \det \boldsymbol{\Omega}_i - \frac{1}{2} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta})^T \boldsymbol{\Omega}_i^{-1} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta}),
$$
where
$$
    \boldsymbol{\Omega}_i = \sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T = \sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \mathbf{L} \mathbf{L}^T \mathbf{Z}_i^T.
$$
Because the variance component parameter $\boldsymbol{\Sigma}$ has to be positive semidefinite, we prefer to use its Cholesky factor $\mathbf{L}$ as optimization variable. 

Given $m$ independent data tuples $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$, $i=1,\ldots,m$, we seek the maximum likelihood estimate (MLE) by maximizing the log-likelihood
$$
\ell(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2) = \sum_{i=1}^m \ell_i(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2).
$$
In this assignment, we use the nonlinear programming (NLP) approach for optimization. In HW7, we will derive an EM (expectation-maximization) algorithm for the same problem. There is also an MM (minorization-maximization) algorithm for the same problem; see [this article](https://doi.org/10.1080/10618600.2018.1529601).

In [5]:
# load necessary packages; make sure install them first
using BenchmarkTools, CSV, DataFrames, DelimitedFiles, Distributions
using Ipopt, LinearAlgebra, MathOptInterface, MixedModels, NLopt
using PrettyTables, Random, RCall

const MOI = MathOptInterface

MathOptInterface

## Q1. (Optional, 30 bonus pts) Derivatives

NLP optimization solvers expect users to provide at least a function for evaluating objective value. If users can provide further information such as gradient and Hessian, the NLP solvers will be more stable and converge faster. Automatic differentiation tools are becoming more powerful but cannot apply to all problems yet.

1. Show that the gradient of $\ell_i$ is
\begin{aligned}
\nabla_{\boldsymbol{\beta}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &= \mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i, \\
\nabla_{\sigma^2} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &= - \frac{1}{2} \operatorname{tr} (\boldsymbol{\Omega}_i^{-1}) + \frac{1}{2} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i, \\
\frac{\partial}{\partial \mathbf{L}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &= - \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} + \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L},
\end{aligned}
where $\mathbf{r}_i = \mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}$. 

2. Derive the observed information matrix and the expected (Fisher) information matrix.

If you need a refresher on multivariate calculus, my [Biostat 216 lecture notes](https://ucla-biostat216-2019fall.github.io/slides/16-matrixcalc/16-matrixcalc.html) may be helpful.


**Solution**

Use the chain rule in 1.2 of 216 lecture notes basically. Note that $D\phi(\mathbf{x}) = \nabla\phi(\mathbf{x})^{\color{red}T}$ when $\phi$ is a scalar function and $\mathbf{x}$ is a vector.

1. Gradient w.r.t $\boldsymbol{\beta}$.
$$
\begin{aligned}
\nabla_{\boldsymbol{\beta}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)
&= -\frac{1}{2}\nabla_{\boldsymbol{\beta}}\mathbf{r}_i \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i\\
&= -\frac{1}{2}\left(D_{\mathbf{r}_i}\mathbf{r}_i \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \cdot D_{\boldsymbol{\beta}}(\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta})\right)^T\\
&= -\frac{1}{2}\left({2\mathbf{r}_i}^T\boldsymbol{\Omega}_i^{-1} \cdot (-\mathbf{X}_i) \right)^T
\quad\because D_{\boldsymbol{\beta}}(\mathbf{X}_i \boldsymbol{\beta}) = \mathbf{X}_i\\
&= \mathbf{X}_i^T\boldsymbol{\Omega}_i^{-1} \mathbf{r}_i.
\end{aligned}
$$
Gradient w.r.t $\sigma^2$. Using the spectral decomposition $\mathbf{Z}\boldsymbol{\Sigma}\mathbf{Z}^T = \mathbf{T}\boldsymbol{\Lambda}\mathbf{T}^T$,
where $\boldsymbol{\Lambda} = \text{diag}(\lambda_1, \ldots, \lambda_{n_i})$,
$$
\begin{aligned}
\nabla_{\sigma^2}\log\det\boldsymbol{\Omega}_i
&= \nabla_{\sigma^2}\log\det(\mathbf{T}(\sigma^2\mathbf{I}_{n_i} + \boldsymbol{\Lambda})\mathbf{T}^T)\\
&= \nabla_{\sigma^2}\sum_{j=1}^{n_i}\log(\sigma^2 + \lambda_j)
\quad\because\det(\mathbf{T}\mathbf{T}^T) = 1\\
&= \sum_{j=1}^{n_i}(\sigma^2 + \lambda_j)^{-1}\\
&= \text{tr}(\mathbf{T}(\sigma^2\mathbf{I}_{n_i} + \boldsymbol{\Lambda})^{-1}\mathbf{T}^T)
\quad\because\mathbf{T}^T\mathbf{T} = \mathbf{I}_{n_i}\\
&= \text{tr}\left(\boldsymbol{\Omega}_i^{-1}\right).
\end{aligned}
$$
Using (59) in the [Matrix Cook Book](https://www.math.uwaterloo.ca/~hwolkowi/matrixcookbook.pdf):
$$
\frac{\partial\mathbf {Y}^{-1}}{\partial x} = -\mathbf{Y}^{-1}\frac{\partial\mathbf {Y}}{\partial x}\mathbf{Y}^{-1}
$$
for scalar $x$ to obtain
$$
\begin{aligned}
\nabla_{\sigma^2} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i
&= \mathbf{r}_i^T \left(\nabla_{\sigma^2}\boldsymbol{\Omega}_i^{-1}\right)\mathbf{r}_i\\
&= -\mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1}(\nabla_{\sigma^2}\boldsymbol{\Omega}_i)\boldsymbol{\Omega}_i^{-1}\mathbf{r}_i\\
&= -\mathbf{r}_i^T \boldsymbol{\Omega}_i^{-2}\mathbf{r}_i.
\end{aligned}
$$
Therefore
$$
\nabla_{\sigma^2} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) = - \frac{1}{2} \operatorname{tr} (\boldsymbol{\Omega}_i^{-1}) + \frac{1}{2} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i.
$$

The gradient w.r.t $\mathbf{L}$. Consider $\boldsymbol{\Sigma}$ first. By the general chain rule and the vectorization formula,
$$
\begin{aligned}
\text{vec}\frac{\partial}{\partial\boldsymbol{\Sigma}}
\left(\log \det \boldsymbol{\Omega}_i + \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i\right)
&= \left(D_{\boldsymbol{\Omega}_i}\left(\log \det \boldsymbol{\Omega}_i + \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i\right)D_{\boldsymbol{\Sigma}}\boldsymbol{\Omega}_i\right)^T\\
&= 
\left\{\left[\text{vec}\frac{\partial}{\partial\boldsymbol{\Omega}}\left(\log \det \boldsymbol{\Omega}_i
+ \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i\right)\right]^T
\cdot D_{\boldsymbol{\Sigma}}\mathbf{Z}_i\boldsymbol{\Sigma}\mathbf{Z}_i^T\right\}^T
\\
&= 
\left\{\left[\text{vec}(\boldsymbol{\Omega}_i^{-1} - \boldsymbol{\Omega}_i^{-1}\mathbf{r}_i\mathbf{r}_i^T\boldsymbol{\Omega}_i^{-1})\right]^T
\cdot(\mathbf{Z}_i\otimes\mathbf{Z}_i)\right\}^T\\
&= (\mathbf{Z}_i^T\otimes \mathbf{Z}_i^T)
\text{vec}(\boldsymbol{\Omega}_i^{-1} - \boldsymbol{\Omega}_i^{-1}\mathbf{r}_i\mathbf{r}_i^T\boldsymbol{\Omega}_i^{-1})\\
&= \text{vec}(\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i - \mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{r}_i\mathbf{r}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i),
\end{aligned}
$$
and therefore
$$
\begin{aligned}
\frac{\partial}{\partial\boldsymbol{\Sigma}}
\left(\log \det \boldsymbol{\Omega}_i + \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i\right)
&= \mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i - \mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{r}_i\mathbf{r}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i\\
\Longrightarrow\frac{\partial}{\partial\boldsymbol{\Sigma}}\ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)
&= -\frac 1 2\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i + \frac 1 2\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{r}_i\mathbf{r}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i.
\end{aligned}
$$
Finally show below:
$$
\begin{aligned}
\frac{\partial}{\partial\mathbf{L}}\ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)
&= 2\frac{\partial}{\partial\boldsymbol{\Sigma}}\ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)\mathbf{L}\\
&= - \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} + \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L}.
\end{aligned}
$$
*Proof*:
$$
\begin{aligned}
D_{\mathbf{L}}\ell_i &= D_{\boldsymbol{\Sigma}}\ell_iD_{\mathbf{L}}\boldsymbol{\Sigma}\\
&= \left(\text{vec}\frac{\partial\ell_i}{\partial\boldsymbol{\Sigma}}\right)^T(\mathbf{I}_{q^2} + \mathbf{K}_{qq})(\mathbf{L}\otimes\mathbf{I}_q)\\
&= \left(\text{vec}\frac{\partial\ell_i}{\partial\boldsymbol{\Sigma}}\right)^T(\mathbf{L}\otimes\mathbf{I}_q)
+ \left(\text{vec}\frac{\partial\ell_i}{\partial\boldsymbol{\Sigma}}\right)^T\mathbf{K}_{qq}(\mathbf{L}\otimes\mathbf{I}_q)\\
&= \left[(\mathbf{L}^T\otimes\mathbf{I}_q)\text{vec}\frac{\partial\ell_i}{\partial\boldsymbol{\Sigma}}\right]^T
+ \left[(\mathbf{L}^T\otimes\mathbf{I}_q)\mathbf{K}_{qq}\text{vec}\frac{\partial\ell_i}{\partial\boldsymbol{\Sigma}}\right]^T
\quad\because \mathbf{K}_{qq}^T = \mathbf{K}_{qq}\\
&= \left[(\mathbf{L}^T\otimes\mathbf{I}_q)\text{vec}\frac{\partial\ell_i}{\partial\boldsymbol{\Sigma}}\right]^T
+ \left[(\mathbf{L}^T\otimes\mathbf{I}_q)\text{vec}\frac{\partial\ell_i}{\partial\boldsymbol{\Sigma}}\right]^T
\quad\because \mathbf{K}_{qq}\text{vec}(\mathbf{A}) = \text{vec}(\mathbf{A}^T) \ \text{but} \ \boldsymbol{\Sigma}^T = \boldsymbol{\Sigma}\ \\
&= 2\left(\text{vec}\frac{\partial\ell_i}{\partial\boldsymbol{\Sigma}}\mathbf{L}\right)^T\\
&= 2D_{\boldsymbol{\Sigma}}(\ell_i\mathbf{L}).
\end{aligned}
$$


2. The Hessian is crazy! The following three blocks (Jacobian) are straightforward:
$$
\begin{aligned}
\nabla^2_{\boldsymbol{\beta}, \boldsymbol{\beta}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)
&= \nabla_{\boldsymbol{\beta}^T}\mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i
= -\mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1}\mathbf{X}_i \in\mathbb{R}^{p\times p} \\
\nabla^2_{\boldsymbol{\beta}, \sigma^2} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)
&= \nabla_{\sigma^2}\mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i
= -\mathbf{X}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i
= \left(\nabla^2_{\sigma^2, \boldsymbol{\beta}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)\right)^T\in\mathbb{R}^{p\times 1}\\
\nabla^2_{\sigma^2, \sigma^2}\ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)
&= - \frac{1}{2} \operatorname{tr} (\nabla_{\sigma^2}\boldsymbol{\Omega}_i^{-1}) + \frac{1}{2} \mathbf{r}_i^T (\nabla_{\sigma^2}\boldsymbol{\Omega}_i^{-2})\mathbf{r}_i\\
&= \frac{1}{2} \operatorname{tr}(\boldsymbol{\Omega}_i^{-1}(\nabla_{\sigma^2}\boldsymbol{\Omega}_i)\boldsymbol{\Omega}_i^{-1})
+ \frac{1}{2} \mathbf{r}_i^T \left[(\nabla_{\sigma^2}\boldsymbol{\Omega}_i^{-1})\boldsymbol{\Omega}_i^{-1} + \boldsymbol{\Omega}_i^{-1}(\nabla_{\sigma^2}\boldsymbol{\Omega}_i^{-1})\right]\mathbf{r}_i\\
&= \frac{1}{2} \operatorname{tr}(\boldsymbol{\Omega}_i^{-2})
-\mathbf{r}_i^T \boldsymbol{\Omega}_i^{-3}\mathbf{r}_i\in\mathbb{R}
\end{aligned}
$$
The remaining blocks are challenging to me. Note $\mathbf{K}_{qq}(\mathbf{L}\otimes\mathbf{I}_q) = (\mathbf{I}_q\otimes\mathbf{L})\mathbf{K}_{qq}$ (very important!).
$$
\begin{aligned}
\nabla^2_{\boldsymbol{\beta}, \text{vec}\mathbf{L}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)
&= \frac{\partial}{\partial(\text{vec}\mathbf{L})^T}\mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i\\
&= D_{\mathbf{L}}\mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i\\
&= D_{\boldsymbol{\Omega}_i}\mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i\cdot
D_{\boldsymbol{\Sigma}}\boldsymbol{\Omega}_i D_{\mathbf{L}}\boldsymbol{\Sigma}\\
&= (\mathbf{r}_i^T\otimes\mathbf{X}_i^T)(-\boldsymbol{\Omega}_i^{-1}\otimes\boldsymbol{\Omega}_i^{-1})(\mathbf{Z}_i\otimes\mathbf{Z}_i)
(\mathbf{I}_{q^2} + \mathbf{K}_{qq})(\mathbf{L}\otimes\mathbf{I}_q)\\
&= -(\mathbf{r}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i\otimes\mathbf{X}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i)(\mathbf{I}_{q^2} + \mathbf{K}_{qq})(\mathbf{L}\otimes\mathbf{I}_q)\\
&= -(\mathbf{r}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i\otimes\mathbf{X}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i)(\mathbf{L}\otimes\mathbf{I}_q) - (\mathbf{r}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i\otimes\mathbf{X}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i)\mathbf{K}_{qq}(\mathbf{L}\otimes\mathbf{I}_q)\\
&= -(\mathbf{r}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i\mathbf{L}\otimes\mathbf{X}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i) - (\mathbf{r}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i\otimes\mathbf{X}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i\mathbf{L})\mathbf{K}_{qq}
\in\mathbb{R}^{p\times q^2},\\
&= \left(\nabla^2_{\text{vec}\mathbf{L}, \boldsymbol{\beta}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)\right)^T,\\
\nabla^2_{\sigma^2, \text{vec}\mathbf{L}}\ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)
&= - \frac{1}{2}\left[D_{\mathbf{L}}\operatorname{tr} (\boldsymbol{\Omega}_i^{-1}) - D_{\mathbf{L}}\mathbf{r}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i\right]\\
&= - \frac{1}{2}\left[D_{\boldsymbol{\Omega}_i}\operatorname{tr} (\boldsymbol{\Omega}_i^{-1}) - D_{\boldsymbol{\Omega}_i}\mathbf{r}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i\right]
D_{\boldsymbol{\Sigma}}\boldsymbol{\Omega}_i D_{\mathbf{L}}\boldsymbol{\Sigma}\\
&= - \frac{1}{2}\left[-(\text{vec}(\boldsymbol{\Omega}_i^{-2}))^T - (\text{vec}(2\boldsymbol{\Omega}_i^{-1}\mathbf{r}_i\mathbf{r}_i^T))^T(-\boldsymbol{\Omega}_i^{-1}\otimes \boldsymbol{\Omega}_i^{-1})\right]
(\mathbf{Z}_i\otimes\mathbf{Z}_i) D_{\mathbf{L}}\boldsymbol{\Sigma}\\
&= - \frac{1}{2}\left[-(\text{vec}(\boldsymbol{\Omega}_i^{-2}))^T(\mathbf{Z}_i\otimes\mathbf{Z}_i) + (\text{vec}(2\boldsymbol{\Omega}_i^{-1}\mathbf{r}_i\mathbf{r}_i^T))^T(\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i\otimes \boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i)\right]
 D_{\mathbf{L}}\boldsymbol{\Sigma}\\
 &= - \frac{1}{2}\left[-(\text{vec}(\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-2}\mathbf{Z}_i))^T + (\text{vec}(2\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-2}\mathbf{r}_i\mathbf{r}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i))^T\right]
 (\mathbf{I}_{q^2} + \mathbf{K}_{qq})(\mathbf{L}\otimes\mathbf{I}_q)\\
 &= - \frac{1}{2}\left[-2(\text{vec}(\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-2}\mathbf{Z}_i\mathbf{L}))^T + 2(\text{vec}(\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-2}\mathbf{r}_i\mathbf{r}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i\mathbf{L} + \mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{r}_i\mathbf{r}_i^T\boldsymbol{\Omega}_i^{-2}\mathbf{Z}_i\mathbf{L}))^T\right]\\
 &= \left(\text{vec}(\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-2}\mathbf{Z}_i\mathbf{L} - \mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-2}\mathbf{r}_i\mathbf{r}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i\mathbf{L} - \mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{r}_i\mathbf{r}_i^T\boldsymbol{\Omega}_i^{-2}\mathbf{Z}_i\mathbf{L})\right)^T
 \in\mathbb{R}^{1\times q^2}.
\end{aligned}
$$
The above second equation can be easily verified by considering the transpose block:
$$
\begin{aligned}
\nabla^2_{\text{vec}\mathbf{L}, \sigma^2} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)
&= \text{vec} \left(- \mathbf{Z}_i^T (\nabla_{\sigma^2}\boldsymbol{\Omega}_i^{-1})\mathbf{Z}_i \mathbf{L}
+ \left[\mathbf{Z}_i^T (\nabla_{\sigma^2}\boldsymbol{\Omega}_i^{-1})\mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} + \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T (\nabla_{\sigma^2}\boldsymbol{\Omega}_i^{-1}) \mathbf{Z}_i \mathbf{L}\right]\right)\\
&= \text{vec}\left(\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-2}\mathbf{Z}_i \mathbf{L}\right)
- \text{vec}\left(\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-2}\mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} + \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-2}\mathbf{Z}_i \mathbf{L}\right)\\
&= \text{vec}(\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-2}\mathbf{Z}_i\mathbf{L} - \mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-2}\mathbf{r}_i\mathbf{r}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i\mathbf{L} - \mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{r}_i\mathbf{r}_i^T\boldsymbol{\Omega}_i^{-2}\mathbf{Z}_i\mathbf{L})
\in\mathbb{R}^{q^2\times 1}\\
&= \left(\nabla^2_{\sigma^2, \text{vec}\mathbf{L}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)\right)^T.
\end{aligned}
$$

The last block matrix should be the most complicated so let me keep $D_{\mathbf{L}}\boldsymbol{\Sigma}$:
$$
\begin{aligned}
\nabla^2_{\text{vec}\mathbf{L}, \text{vec}\mathbf{L}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)
&= \frac{\partial\text{vec}(- \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} + \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L})}{\partial(\text{vec}\mathbf{L})^T}\\
&= D_{\mathbf{L}}(- \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} + \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L})\\
&= D_{\boldsymbol{\Omega}_i}(- \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} + \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L})
D_{\boldsymbol{\Sigma}}\boldsymbol{\Omega}_i D_{\mathbf{L}}\boldsymbol{\Sigma}\\
&= D_{\boldsymbol{\Omega}_i^{-1}}(- \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} + \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L})
D_{\boldsymbol{\Omega}_i}\boldsymbol{\Omega}_i^{-1}\cdot D_{\boldsymbol{\Sigma}}\boldsymbol{\Omega}_i D_{\mathbf{L}}\boldsymbol{\Sigma}\\
&= (- \mathbf{L}^T\mathbf{Z}_i^T \otimes\mathbf{Z}_i^T + 
(\mathbf{L}^T\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T\otimes\mathbf{Z}_i^T + \mathbf{L}^T\mathbf{Z}_i^T\otimes \mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T))
(-\boldsymbol{\Omega}_i^{-1}\otimes \boldsymbol{\Omega}_i^{-1})(\mathbf{Z}_i\otimes\mathbf{Z}_i)D_{\mathbf{L}}\boldsymbol{\Sigma}\\
&= (\mathbf{L}^T\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i \otimes\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i -
\mathbf{L}^T\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i\otimes\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i - \mathbf{L}^T\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i\otimes \mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i)
D_{\mathbf{L}}\boldsymbol{\Sigma}
\end{aligned}
$$

The Hessian for $i$-th datum can be expressed as
$$
\mathbf{H}(\ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)) = 
\begin{bmatrix}
\nabla^2_{\boldsymbol{\beta}, \boldsymbol{\beta}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &
\nabla^2_{\boldsymbol{\beta}, \sigma^2} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &
\nabla^2_{\boldsymbol{\beta}, \text{vec}\mathbf{L}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)\\
\nabla^2_{\sigma^2, \boldsymbol{\beta}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &
\nabla^2_{\sigma^2, \sigma^2} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &
\nabla^2_{\sigma^2, \text{vec}\mathbf{L}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)\\
\nabla^2_{\text{vec}\mathbf{L}, \boldsymbol{\beta}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &
\nabla^2_{\text{vec}\mathbf{L}, \sigma^2} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &
\nabla^2_{\text{vec}\mathbf{L}, \text{vec}\mathbf{L}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)
\end{bmatrix}
\in\mathbb{R}^{(p+1+q^2)\times(p+1+q^2)}
$$
and the observed information matrix and the expected (Fisher) information matrix are
$$
-\mathbf{H}(\ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)), \quad
E\left[-\mathbf{H}(\ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2))\right],
$$
respectively. Taking expectation makes some terms canceled out and simplified due to $E(\mathbf{r}_i) = \mathbf{0}$ and $E(\mathbf{r}_i\mathbf{r}_i^T) = \boldsymbol{\Omega}_i$. Specifically,
$$
\begin{aligned}
E\left[-\nabla^2_{\boldsymbol{\beta}, \boldsymbol{\beta}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)\right]
&= E(\mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1}\mathbf{X}_i) = \mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1}\mathbf{X}_i\\
E\left[-\nabla^2_{\boldsymbol{\beta}, \sigma^2} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)\right]
&= E(\mathbf{X}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i) = \mathbf{0},\\
E\left[-\nabla^2_{\boldsymbol{\beta}, \text{vec}\mathbf{L}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)\right]
&= E\left[-D_{\mathbf{L}}\mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i\right]
= \mathbf{O}\\
E\left[-\nabla^2_{\sigma^2, \sigma^2} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)\right]
&= E\left[\frac{1}{2} \operatorname{tr}(\boldsymbol{\Omega}_i^{-2})
-\mathbf{r}_i^T \boldsymbol{\Omega}_i^{-3}\mathbf{r}_i\right] = -\frac{1}{2} \operatorname{tr}(\boldsymbol{\Omega}_i^{-2})\\
&\because E(\mathbf{r}_i^T \boldsymbol{\Omega}_i^{-3}\mathbf{r}_i) = E\left[\text{tr}(\boldsymbol{\Omega}_i^{-3}\mathbf{r}_i\mathbf{r}_i^T)\right] = \text{tr}(\boldsymbol{\Omega}_i^{-3}E(\mathbf{r}_i\mathbf{r}_i^T)) = \text{tr}(\boldsymbol{\Omega}_i^{-2})\\
E\left[-\nabla^2_{\text{vec}\mathbf{L}, \sigma^2} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)\right]
&= -\text{vec}(\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-2}\mathbf{Z}_i\mathbf{L} - \mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-2}E(\mathbf{r}_i\mathbf{r}_i^T)\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i\mathbf{L} - \mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}E(\mathbf{r}_i\mathbf{r}_i^T)\boldsymbol{\Omega}_i^{-2}\mathbf{Z}_i\mathbf{L})\\
&= -\text{vec}(\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-2}\mathbf{Z}_i\mathbf{L} - \mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-2}\mathbf{Z}_i\mathbf{L} - \mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-2}\mathbf{Z}_i\mathbf{L})\\
&= \text{vec}(\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-2}\mathbf{Z}_i\mathbf{L})\\
E\left[-\nabla^2_{\text{vec}\mathbf{L}, \text{vec}\mathbf{L}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)\right]
&= -(\mathbf{L}^T\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i \otimes\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i -
\mathbf{L}^T\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i\otimes\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i - \mathbf{L}^T\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i\otimes \mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i)
D_{\mathbf{L}}\boldsymbol{\Sigma}\\
&= (\mathbf{L}^T\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i \otimes\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i)
D_{\mathbf{L}}\boldsymbol{\Sigma}\\
&= (\mathbf{L}^T\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i \otimes\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i)
(\mathbf{I}_{q^2} + \mathbf{K}_{qq})(\mathbf{L}\otimes\mathbf{I}_q)\\
&= (\mathbf{L}^T\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i \otimes\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i)(\mathbf{L}\otimes\mathbf{I}_q)
+ (\mathbf{L}^T\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i \otimes\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i)\mathbf{K}_{qq}(\mathbf{L}\otimes\mathbf{I}_q)\\
&= (\mathbf{L}^T\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i\mathbf{L} \otimes\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i)
+ (\mathbf{L}^T\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i \otimes\mathbf{Z}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{Z}_i\mathbf{L})\mathbf{K}_{qq}.
\end{aligned}
$$
I learned a lot!

## Q2. (20 pts) Objective and gradient evaluator for a single datum

We expand the code from HW3 to evaluate both objective and gradient. I provide my code for HW3 below as a starting point. You do _not_ have to use this code. If your come up faster code, that's even better. 

**Solution**

We first rewrite the three gradient expressions for efficient computation using
$
\mathbf{\Omega}_i^{-1}
= \sigma^{-2} \mathbf I_{n_i} - \sigma^{-4}\mathbf{Z}_i\mathbf{L}(\mathbf{M}\mathbf{M}^T)^{-1}\mathbf{L}^T\mathbf{Z}_i^T,
$
where $\mathbf{M}\mathbf{M}^T = \mathbf{I}_q + \sigma^{-2}\mathbf{L}^T\mathbf{Z}_i^T\mathbf{Z}_i\mathbf{L}$:
$$
\begin{aligned}
\nabla_{\boldsymbol{\beta}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)
&= \mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i\\
&= \sigma^{-2}\mathbf{X}_i^T\mathbf{r}_i -
\sigma^{-4}\mathbf{X}_i^T\mathbf{Z}_i\mathbf{L}(\mathbf{M}\mathbf{M}^T)^{-1}\mathbf{L}^T\mathbf{Z}_i^T\mathbf{r}_i\\
&= \sigma^{-2}\mathbf{X}_i^T\mathbf{r}_i
- \sigma^{-4}\mathbf{X}_i^T\mathbf{Z}_i\mathbf{L}\mathbf{M}^{-T}\mathbf{M}^{-1}\mathbf{L}^T\mathbf{Z}_i^T\mathbf{r}_i
\end{aligned}
$$
where $\mathbf{X}_i^T\mathbf{r}_i$ and $\mathbf{Z}_i^T\mathbf{r}_i$ are stored in `storage_p` and `storage_q`, respectively.

For $\nabla_{\sigma^2} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)$, consider each term separately and merge them:
$$
\begin{aligned}
\operatorname{tr} (\boldsymbol{\Omega}_i^{-1})
&= \operatorname{tr} \left(\sigma^{-2} \mathbf I_{n_i} - \sigma^{-4}\mathbf{Z}_i\mathbf{L}(\mathbf{M}\mathbf{M}^T)^{-1}\mathbf{L}^T\mathbf{Z}_i^T\right)\\
&= n_i\sigma^{-2} - \sigma^{-4}\operatorname{tr} \left(\mathbf{Z}_i\mathbf{L}(\mathbf{M}\mathbf{M}^T)^{-1}\mathbf{L}^T\mathbf{Z}_i^T\right)\\
&= n_i\sigma^{-2} - \sigma^{-4}\operatorname{tr} \left(\mathbf{Z}_i^T\mathbf{Z}_i\mathbf{L}(\mathbf{M}\mathbf{M}^T)^{-1}\mathbf{L}^T\right),\\
&= n_i\sigma^{-2} - \sigma^{-4}\operatorname{dot} \left(\mathbf{Z}_i^T\mathbf{Z}_i, \mathbf{L}(\mathbf{M}\mathbf{M}^T)^{-1}\mathbf{L}^T\right),\\
\mathbf{r}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i
&= \|\boldsymbol{\Omega}_i^{-1} \mathbf{r}_i\|_2^2\\
&= \left\|\left(\sigma^{-2} \mathbf I_{n_i} - \sigma^{-4}\mathbf{Z}_i\mathbf{L}(\mathbf{M}\mathbf{M}^T)^{-1}\mathbf{L}^T\mathbf{Z}_i^T\right)\mathbf{r}_i\right\|_2^2\\
&= \sigma^{-4}\left\|\left(\mathbf I_{n_i} - \sigma^{-2}\mathbf{Z}_i\mathbf{L}(\mathbf{M}\mathbf{M}^T)^{-1}\mathbf{L}^T\mathbf{Z}_i^T\right)\mathbf{r}_i\right\|_2^2\\
&= \sigma^{-4}\left[\mathbf{r}_i^T\mathbf{r}_i -2\sigma^{-2}\mathbf{Z}_i\mathbf{L}(\mathbf{M}\mathbf{M}^T)^{-1}\mathbf{L}^T\mathbf{Z}_i^T\mathbf{r}_i
+ \sigma^{-4}\mathbf{r}_i^T\mathbf{Z}_i\mathbf{L}(\mathbf{M}\mathbf{M}^T)^{-1}\mathbf{L}^T\mathbf{Z}_i^T\mathbf{Z}_i\mathbf{L}(\mathbf{M}\mathbf{M}^T)^{-1}\mathbf{L}^T\mathbf{Z}_i^T\mathbf{r}_i\right]\\
&= \sigma^{-4}\left[\mathbf{r}_i^T\mathbf{r}_i - \sigma^{-2}\left(\mathbf{L}(\mathbf{M}\mathbf{M}^T)^{-1}\mathbf{L}^T\mathbf{Z}_i^T\mathbf{r}_i\right)^T
\left(2\mathbf{Z}_i^T\mathbf{r}_i - \sigma^{-2}\mathbf{Z}_i^T\mathbf{Z}_i\mathbf{L}(\mathbf{M}\mathbf{M}^T)^{-1}\mathbf{L}^T\mathbf{Z}_i^T\mathbf{r}_i\right)\right]
\end{aligned}
$$
and therefore
$$
\begin{aligned}
\nabla_{\sigma^2} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)
&= - \frac{1}{2} \operatorname{tr} (\boldsymbol{\Omega}_i^{-1}) + \frac{1}{2} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i\\
&= - \frac{n_i\sigma^{-2}}{2} + \frac{\sigma^{-4}}{2}\operatorname{dot} \left(\mathbf{Z}_i^T\mathbf{Z}_i, \mathbf{L}(\mathbf{M}\mathbf{M}^T)^{-1}\mathbf{L}^T\right)\\
&\quad + \frac{\sigma^{-4}}{2}\left[\mathbf{r}_i^T\mathbf{r}_i - \sigma^{-2}\left(\mathbf{L}(\mathbf{M}\mathbf{M}^T)^{-1}\mathbf{L}^T\mathbf{Z}_i^T\mathbf{r}_i\right)^T
\left(2\mathbf{Z}_i^T\mathbf{r}_i - \sigma^{-2}\mathbf{Z}_i^T\mathbf{Z}_i\mathbf{L}(\mathbf{M}\mathbf{M}^T)^{-1}\mathbf{L}^T\mathbf{Z}_i^T\mathbf{r}_i\right)\right],
\end{aligned}
$$
where $\mathbf{r}_i^T\mathbf{r}_i = \mathbf{y}_i^T\mathbf{y}_i + \boldsymbol{\beta}^T(\mathbf{X}_i^T\mathbf{X}_i\boldsymbol{\beta} - 2\mathbf{X}_i^T\mathbf{y}_i)$.

Lastly,
$$
\begin{aligned}
\frac{\partial}{\partial \mathbf{L}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)
&= - \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i\mathbf{L} + \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i\mathbf{L}\\
&= - \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i\mathbf{L} + \left(\mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i\right) \left(\mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i\right)^T\mathbf{L}\\
&= - \mathbf{Z}_i^T\left[\sigma^{-2} \mathbf I_{n_i} - \sigma^{-4}\mathbf{Z}_i\mathbf{L}(\mathbf{M}\mathbf{M}^T)^{-1}\mathbf{L}^T\mathbf{Z}_i^T\right]\mathbf{Z}_i\mathbf{L}
+ \left(\mathbf{Z}_i^T \left[\sigma^{-2} \mathbf I_{n_i} - \sigma^{-4}\mathbf{Z}_i\mathbf{L}(\mathbf{M}\mathbf{M}^T)^{-1}\mathbf{L}^T\mathbf{Z}_i^T\right] \mathbf{r}_i\right) \left(\mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i\right)^T\mathbf{L}\\
&= - \left[\sigma^{-2} \mathbf I_{q} - \sigma^{-4}\mathbf{Z}_i^T\mathbf{Z}_i\mathbf{L}(\mathbf{M}\mathbf{M}^T)^{-1}\mathbf{L}^T\right] \mathbf{Z}_i^T\mathbf{Z}_i\mathbf{L}
+ \left[\sigma^{-2} \mathbf I_{q} - \sigma^{-4}\mathbf{Z}_i^T\mathbf{Z}_i\mathbf{L}(\mathbf{M}\mathbf{M}^T)^{-1}\mathbf{L}^T\right] \mathbf{Z}_i^T\mathbf{r}_i \left(\mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i\right)^T\mathbf{L}\\
&= -\mathbf{W}_i\mathbf{Z}_i^T\mathbf{Z}_i\mathbf{L} + \mathbf{W}_i\mathbf{Z}_i^T\mathbf{r}_i(\mathbf{W}_i\mathbf{Z}_i^T\mathbf{r}_i)^T\mathbf{L},
\end{aligned}
$$
where $\mathbf{W}_i = \sigma^{-2} \mathbf I_{q} - \sigma^{-4}\mathbf{Z}_i^T\mathbf{Z}_i\mathbf{L}(\mathbf{M}\mathbf{M}^T)^{-1}\mathbf{L}^T$, which is *not* symmetric.

Use my code for `log!()` developed in HW3. I further pre-allocate `storage_q2` and `storage_qq2` for the gradients. Note that a few lines exceed 80 characters, but remember that the criterion for the course is 92 characters.

In [6]:
# define a type that holds an LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y          :: Vector{T}
    X          :: Matrix{T}
    Z          :: Matrix{T}
    # arrays for holding gradient
    ∇β         :: Vector{T}
    ∇σ²        :: Vector{T}
    ∇Σ         :: Matrix{T}  
    ∇L         :: Matrix{T} # added
    # working arrays
    # TODO: whatever intermediate arrays you may want to pre-allocate
    yty        :: T
    xty        :: Vector{T}
    zty        :: Vector{T}
    storage_p  :: Vector{T}
    storage_q  :: Vector{T}
    storage_q2 :: Vector{T} # added
    xtx        :: Matrix{T}
    ztx        :: Matrix{T}
    ztz        :: Matrix{T}
    storage_qq :: Matrix{T}
    storage_qq2 :: Matrix{T} # added
end

"""
    LmmObs(y::Vector, X::Matrix, Z::Matrix)

Create an LMM datum of type `LmmObs`.
"""
function LmmObs(
    y          ::Vector{T}, 
    X          ::Matrix{T}, 
    Z          ::Matrix{T}
    ) where T <: AbstractFloat
    n, p, q    = size(X, 1), size(X, 2), size(Z, 2)    
    ∇β         = Vector{T}(undef, p)
    ∇σ²        = Vector{T}(undef, 1)
    ∇Σ         = Matrix{T}(undef, q, q)
    ∇L         = Matrix{T}(undef, q, q) # added
    yty        = abs2(norm(y))
    xty        = transpose(X) * y
    zty        = transpose(Z) * y    
    storage_p  = Vector{T}(undef, p)
    storage_q  = Vector{T}(undef, q)
    storage_q2 = Vector{T}(undef, q) # added
    xtx        = transpose(X) * X
    ztx        = transpose(Z) * X
    ztz        = transpose(Z) * Z
    storage_qq = similar(ztz)
    storage_qq2 = similar(ztz) # added
    LmmObs(y, X, Z, ∇β, ∇σ², ∇Σ, ∇L,
        yty, xty, zty, storage_p, storage_q, storage_q2, 
        xtx, ztx, ztz, storage_qq, storage_qq2)
end

"""
    logl!(obs::LmmObs, β, L, σ², needgrad=false)

Evaluate the log-likelihood of a single LMM datum at parameter values `β`, `L`, 
and `σ²`. If `needgrad==true`, then `obs.∇β`, `obs.∇Σ`, and `obs.σ² are filled 
with the corresponding gradient.
"""
function logl!(
    obs      :: LmmObs{T}, 
    β        :: Vector{T}, 
    L        :: Matrix{T}, 
    σ²       :: T,
    needgrad :: Bool = true
    ) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)
    σ²inv   = inv(σ²)
    ####################
    # Evaluate objective
    ####################    
    ## 1st term = -(n/2) log(2πσ²) ----------------------------------------- ##
    LogLik = -(n//2) * log(2π * σ²)

    ## 2nd term = -logdet(M), where M = L'Z'ZL / σ² + I -------------------- ##
    mul!(obs.storage_qq, transpose(L), obs.ztz) # L'Z'Z
    BLAS.trmm!('R', 'L', 'N', 'N', σ²inv, L, obs.storage_qq) # (L'Z'Z)L / σ²
    @inbounds for j in 1:q
        obs.storage_qq[j, j] += 1.0 # L'Z'ZL / σ² + I
    end
    LAPACK.potrf!('L', obs.storage_qq) # M = cholesky!(Symmetric(obs.storage_qq, :L))
    LogLik -= logdet(LowerTriangular(obs.storage_qq)) # logdet(M)

    ## 3rd term = -1/2 (||y||^2 + β'(X'Xβ - 2X'y)) / σ² -------------------- ##
    obs.storage_p .= obs.xty # X'y
    BLAS.symv!('U', T(1), obs.xtx, β, T(-2), obs.storage_p) # X'Xβ - 2X'y
    LogLik -= (1//2) * (obs.yty + dot(β, obs.storage_p)) * σ²inv

    ## 4th term = 1/2 ||M⁻¹(L'Z'y - L'Z'Xβ)||² / (σ²)² --------------------- ##
    obs.storage_q .= obs.zty                                # Z'y
    BLAS.gemv!('N', T(-1), obs.ztx, β, T(1), obs.storage_q) # Z'y - Z'Xβ = Z'r
    BLAS.trmv!('L', 'T', 'N', L, obs.storage_q)             # L'Z'r
    BLAS.trsv!('L', 'N', 'N', obs.storage_qq, obs.storage_q) # M⁻¹L'Z'r
    LogLik += (1//2) * abs2(norm(obs.storage_q)) * σ²inv^2
    
    ###################
    # Evaluate gradient
    ###################   
    if needgrad
        # TODO: fill ∇β, ∇Σ, ∇L, ∇σ² by gradients
        # ∇β ---------------------------------------------------------------- # 
        BLAS.symv!('U', T(-1), obs.xtx, β, T(1), copy!(obs.∇β, obs.xty)) # X'y - X'Xβ = X'r
        BLAS.trsv!('L', 'T', 'N', obs.storage_qq, obs.storage_q) # M⁻ᵀ(M⁻¹L'Z'r)
        BLAS.trmv!('L', 'N', 'N', L, obs.storage_q)              # L(M⁻ᵀM⁻¹L'Z'r)
        BLAS.gemv!('T', -σ²inv^2, obs.ztx, obs.storage_q, σ²inv, obs.∇β) # ∇β

        # 2∇Σ (not ∇Σ) ------------------------------------------------------ #
        ## 1st term = -WZ'Z ------------------------------------------------ ##
        BLAS.trmm!('R', 'L', 'N', 'N', T(1), L, copy!(obs.storage_qq2, obs.ztz)) # Z'ZL
        BLAS.trsm!('R', 'L', 'T', 'N', T(1), obs.storage_qq, obs.storage_qq2) # Z'ZLM⁻ᵀ
        BLAS.trsm!('R', 'L', 'N', 'N', T(1), obs.storage_qq, obs.storage_qq2) # Z'ZLM⁻ᵀM⁻¹
        BLAS.trmm!('R', 'L', 'T', 'N', T(-σ²inv), L, obs.storage_qq2)         # -σ⁻²Z'ZLM⁻ᵀM⁻¹L'
        @inbounds for j in 1:q
            obs.storage_qq2[j, j] += 1.0 # I - σ⁻²Z'ZLM⁻ᵀM⁻¹L
        end
        obs.storage_qq2 .*= σ²inv # σ⁻²I - σ⁻⁴Z'ZLM⁻ᵀM⁻¹L' = W (asymmetric)
        BLAS.symm!('R', 'U', T(-1), obs.ztz, obs.storage_qq2 , T(0), obs.∇Σ) # -WZ'Z
        ## 2nd term = (WZtr)(WZtr)' ---------------------------------------- ##
        BLAS.gemv!('N', T(-1), obs.ztx, β, T(1), copy!(obs.storage_q, obs.zty)) # Z'r
        BLAS.gemv!('N', T(1), obs.storage_qq2, obs.storage_q, T(0), obs.storage_q2) # WZ'r
        ### sum of the two terms after outer product
        BLAS.ger!(T(1), obs.storage_q2, obs.storage_q2, obs.∇Σ) # 2∇Σ
        # ∇L ---------------------------------------------------------------- #
        BLAS.trmm!('R', 'L', 'N', 'N', T(1), L, copy!(obs.∇L, obs.∇Σ)) # 2∇ΣL

        # ∇σ² --------------------------------------------------------------- #
        ## 1st term (constant) --------------------------------------------- ##
        obs.∇σ² .= -(n//2) * σ²inv
        ## 2nd term (dot product) ------------------------------------------ ##
        BLAS.trsm!('R', 'L', 'T', 'N', T(1), obs.storage_qq, copy!(obs.storage_qq2, L)) # LM⁻ᵀ
        BLAS.trsm!('R', 'L', 'N', 'N', T(1), obs.storage_qq, obs.storage_qq2) # LM⁻ᵀM⁻¹
        BLAS.trmm!('R', 'L', 'T', 'N', T(1), L, obs.storage_qq2) # LM⁻ᵀM⁻¹L'
        obs.∇σ² .+= (1//2) * σ²inv^2 * dot(obs.ztz, obs.storage_qq2) # σ⁻⁴tr(Z'ZLM⁻ᵀM⁻¹L')
        ## 3rd term (norm) ------------------------------------------------- ##
        BLAS.symv!('L', T(1), obs.storage_qq2, obs.storage_q, T(0), obs.storage_q2) # LM⁻ᵀM⁻¹L'Z'r
        BLAS.symv!('L', -σ²inv, obs.ztz, obs.storage_q2, T(2), obs.storage_q)
        obs.∇σ² .+= (1//2) * σ²inv^2 * (obs.yty + dot(β, obs.storage_p) # r'r
                    - σ²inv * dot(obs.storage_q2, obs.storage_q)) # remaining term
    end  
    ###################
    # Return
    ###################        
    return LogLik
end

logl!

It is a good idea to test correctness and efficiency of the single datum objective/gradient evaluator here. First generate the same data set as in [HW3](https://ucla-biostat-257.github.io/2024spring/hw/hw3/hw03.html).

In [7]:
Random.seed!(257)

# dimension
n, p, q = 2000, 5, 3
# predictors
X  = [ones(n) randn(n, p - 1)]
Z  = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I # compound symmetry 
L  = Matrix(cholesky(Symmetric(Σ)).L)
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form the LmmObs object
obs = LmmObs(y, X, Z);

### Correctness

In [8]:
@show logl = logl!(obs, β, L, σ², true)
@show obs.∇β
@show obs.∇σ²
@show obs.∇Σ
@show obs.∇L;

logl = logl!(obs, β, L, σ², true) = -3256.1793358058385
obs.∇β = [0.2669810805700901, 41.61418337067374, -34.346649623127014, 36.108985107075085, 27.913948208793503]
obs.∇σ² = [1.6283715138490606]
obs.∇Σ = [-0.9464482950697549 0.057792444809472696 -0.3024412763919055; 0.05779244480946695 -1.0008716491712542 0.2845116557145112; -0.30244127639187135 0.28451165571452075 1.1700409272598846]
obs.∇L = [-0.9709131782279983 0.030145913774881482 -0.2996791977474491; -0.013843554536207352 -0.9701196695228238 0.2819133213280048; -0.15698601809443083 0.38891970710309437 1.1593553981651827]


You will lose all 20 points if following statement throws `AssertionError`.

**Comment**

It is important to note that the values for `obs.∇Σ` by Hua are twice as the actual values because he (and I accordingly) did not multiply it by 0.5.

In [9]:
@assert abs(logl - (-3256.1793358058258)) < 1e-4
@assert norm(obs.∇β - [0.26698108057144054, 41.61418337067327, 
        -34.34664962312689, 36.10898510707527, 27.913948208793144]) < 1e-4
@assert norm(obs.∇Σ - 
    [-0.9464482950697888 0.057792444809492895 -0.30244127639188767; 
        0.057792444809492895 -1.00087164917123 0.2845116557144694; 
        -0.30244127639188767 0.2845116557144694 1.170040927259726]) < 1e-4
@assert abs(obs.∇σ²[1] - (1.6283715138412163)) < 1e-4

### Efficiency

Benchmark for evaluating objective only. This is what we did in HW3.

In [10]:
@benchmark logl!($obs, $β, $L, $σ², false)

BenchmarkTools.Trial: 10000 samples with 140 evaluations.
 Range (min … max):  710.421 ns … 1.074 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     713.686 ns             ┊ GC (median):    0.00%
 Time  (mean ± σ):   714.516 ns ± 6.069 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▄▄▆▆█▇▇▆▆▄▃▂▂▁                                           ▂
  ▄▆███████████████▇██▇▇▇█▇▇▇█▆▇▇▇▅▆▆█▇▇▆█▆▆▆▇▇▇▇█▄▆▆▇▄▅▅▆▅▁▅ █
  710 ns       Histogram: log(frequency) by time       732 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

Benchmark for objective + gradient evaluation.

In [11]:
bm_objgrad = @benchmark logl!($obs, $β, $L, $σ², true)

BenchmarkTools.Trial: 10000 samples with 9 evaluations.
 Range (min … max):  2.574 μs …  4.218 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.625 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.626 μs ± 33.391 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

                      ▁  ▄  ▅  █ █  ▆  ▅  ▂                   
  ▂▁▂▁▁▂▁▁▂▁▁▃▁▁▄▁▁▆▁▁█▁▁█▁▁█▁▁█▁█▁▁█▁▁█▁▁█▁▁▇▁▁▅▁▁▄▁▁▃▁▁▂▁▂ ▃
  2.57 μs        Histogram: frequency by time        2.67 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

My median runt time is 900ns. You will get full credit (10 pts) if the median run time is within 10μs.

In [12]:
#  The points you will get are
clamp(10 / (median(bm_objgrad).time / 1e3) * 10, 0, 10)

10.0

## Q3. LmmModel type

We create a `LmmModel` type to hold all data points and model parameters. Log-likelihood/gradient of a `LmmModel` object is simply the sum of log-likelihood/gradient of individual data points. 

In [13]:
# define a type that holds LMM model (data + parameters)
struct LmmModel{T <: AbstractFloat} <: MOI.AbstractNLPEvaluator
    # data
    data :: Vector{LmmObs{T}}
    # parameters
    β    :: Vector{T}
    L    :: Matrix{T}
    σ²   :: Vector{T}    
    # arrays for holding gradient
    ∇β   :: Vector{T}
    ∇σ²  :: Vector{T}
    ∇L   :: Matrix{T}
    # TODO: add whatever intermediate arrays you may want to pre-allocate
    xty  :: Vector{T}
    ztr2 :: Vector{T}
    storage_2q :: Vector{T} # added to store ztr2 for each i
    xtx  :: Matrix{T}
    ztz2 :: Matrix{T}
    storage_2q2q :: Matrix{T} # added to store ztz2 for each i
end

"""
    LmmModel(data::Vector{LmmObs})

Create an LMM model that contains data and parameters.
"""
function LmmModel(obsvec::Vector{LmmObs{T}}) where T <: AbstractFloat
    # dims
    p    = size(obsvec[1].X, 2)
    q    = size(obsvec[1].Z, 2)
    # parameters
    β    = Vector{T}(undef, p)
    L    = Matrix{T}(undef, q, q)
    σ²   = Vector{T}(undef, 1)    
    # gradients
    ∇β   = similar(β)    
    ∇σ²  = similar(σ²)
    ∇L   = similar(L)
    # intermediate arrays
    xty  = Vector{T}(undef, p)
    ztr2 = Vector{T}(undef, abs2(q))
    storage_2q = similar(ztr2) # added
    xtx  = Matrix{T}(undef, p, p)
    ztz2 = Matrix{T}(undef, abs2(q), abs2(q))
    storage_2q2q = similar(ztz2) # added
    LmmModel(obsvec, β, L, σ², ∇β, ∇σ², ∇L, xty, ztr2, storage_2q, xtx, ztz2, storage_2q2q)
end

"""
    logl!(m::LmmModel, needgrad=false)

Evaluate the log-likelihood of an LMM model at parameter values `m.β`, `m.L`, 
and `m.σ²`. If `needgrad==true`, then `m.∇β`, `m.∇Σ`, and `m.σ² are filled 
with the corresponding gradient.
"""
function logl!(m::LmmModel{T}, needgrad::Bool = false) where T <: AbstractFloat
    logl = zero(T)
    if needgrad
        fill!(m.∇β , 0)
        fill!(m.∇L , 0)
        fill!(m.∇σ², 0)        
    end
    @inbounds for i in 1:length(m.data)
        obs = m.data[i]
        logl += logl!(obs, m.β, m.L, m.σ²[1], needgrad)
        if needgrad
            BLAS.axpy!(T(1), obs.∇β, m.∇β)
            # BLAS.axpy!(T(1), obs.∇Σ, m.∇L) # I defined ∇L
            BLAS.axpy!(T(1), obs.∇L, m.∇L)
            m.∇σ²[1] += obs.∇σ²[1]
        end
    end
    logl
end

logl!

## Q4. (20 pts) Test data

Let's generate a synthetic longitudinal data set to test our algorithm.

In [14]:
Random.seed!(257)

# dimension
m      = 1000 # number of individuals
ns     = rand(1500:2000, m) # numbers of observations per individual
p      = 5 # number of fixed effects, including intercept
q      = 3 # number of random effects, including intercept
obsvec = Vector{LmmObs{Float64}}(undef, m)
# true parameter values
βtrue  = [0.1; 6.5; -3.5; 1.0; 5; zeros(p - 5)]
σ²true = 1.5
σtrue  = sqrt(σ²true)
Σtrue  = Matrix(Diagonal([2.0; 1.2; 1.0; zeros(q - 3)]))
Ltrue  = Matrix(cholesky(Symmetric(Σtrue), Val(true), check=false).L)
# generate data
for i in 1:m
    # first column intercept, remaining entries iid std normal
    X = Matrix{Float64}(undef, ns[i], p)
    X[:, 1] .= 1
    @views Distributions.rand!(Normal(), X[:, 2:p])
    # first column intercept, remaining entries iid std normal
    Z = Matrix{Float64}(undef, ns[i], q)
    Z[:, 1] .= 1
    @views Distributions.rand!(Normal(), Z[:, 2:q])
    # generate y
    y = X * βtrue .+ Z * (Ltrue * randn(q)) .+ σtrue * randn(ns[i])
    # form a LmmObs instance
    obsvec[i] = LmmObs(y, X, Z)
end
# form a LmmModel instance
lmm = LmmModel(obsvec);

For later comparison with other software, we save the data into a text file `lmm_data.csv`. **Do not put this file in Git.** It takes 245.4MB storage.

In [15]:
(isfile("lmm_data.csv") && filesize("lmm_data.csv") == 245369685) || 
open("lmm_data.csv", "w") do io
    p = size(lmm.data[1].X, 2)
    q = size(lmm.data[1].Z, 2)
    # print header
    print(io, "ID,Y,")
    for j in 1:(p-1)
        print(io, "X" * string(j) * ",")
    end
    for j in 1:(q-1)
        print(io, "Z" * string(j) * (j < q-1 ? "," : "\n"))
    end
    # print data
    for i in eachindex(lmm.data)
        obs = lmm.data[i]
        for j in 1:length(obs.y)
            # id
            print(io, i, ",")
            # Y
            print(io, obs.y[j], ",")
            # X data
            for k in 2:p
                print(io, obs.X[j, k], ",")
            end
            # Z data
            for k in 2:q-1
                print(io, obs.Z[j, k], ",")
            end
            print(io, obs.Z[j, q], "\n")
        end
    end
end

true

### Correctness

Evaluate log-likelihood and gradient of whole data set at the true parameter values.

In [16]:
copy!(lmm.β, βtrue)
copy!(lmm.L, Ltrue)
lmm.σ²[1] = σ²true
@show obj = logl!(lmm, true)
@show lmm.∇β
@show lmm.∇σ²
@show lmm.∇L;

obj = logl!(lmm, true) = -2.8400684383699736e6
lmm.∇β = [41.06591670742383, 445.7512035395316, 157.0133992248792, -335.0997736073277, -895.6257448387724]
lmm.∇σ² = [-489.53617303727367]
lmm.∇L = [-3.398257593515736 31.32103842087272 26.736450897355372; 40.4352867299875 61.86377650461247 -75.37427770755431; 37.81105146876211 -82.5683843121727 -56.459925427587095]


Test correctness. You will loss all 20 points if following code throws `AssertError`.

In [17]:
@assert abs(obj - (-2.840068438369969e6)) < 1e-4
@assert norm(lmm.∇β - [41.0659167074073, 445.75120353972426, 
        157.0133992249258, -335.09977360733626, -895.6257448385899]) < 1e-4
@assert norm(lmm.∇L - [-3.3982575935824837 31.32103842086001 26.73645089732865; 
        40.43528672997116 61.86377650461202 -75.37427770754684; 
        37.811051468724486 -82.56838431216435 -56.45992542754974]) < 1e-4
@assert abs(lmm.∇σ²[1] - (-489.5361730382465)) < 1e-4

### Efficiency

Test efficiency.

In [18]:
bm_model = @benchmark logl!($lmm, true)

BenchmarkTools.Trial: 1882 samples with 1 evaluation.
 Range (min … max):  2.631 ms …  4.689 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.641 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.655 ms ± 74.391 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▆█▅▃▂▂                                                      
  ██████████▇▇▇▇▆▄▄▃▆▃▃▁▅▆▃▃▃▃▃▃▃▃▃▁▁▃▁▁▁▁▃▃▁▃▃▁▁▁▁▁▁▁▃▁▁▁▁▆ █
  2.63 ms      Histogram: log(frequency) by time     2.99 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

My median run time is 1.4ms. You will get full credit if your median run time is within 10ms. The points you will get are

In [19]:
clamp(10 / (median(bm_model).time / 1e6) * 10, 0, 10)

10.0

### Memory

You will lose 1 point for each 100 bytes memory allocation. So the points you will get is

In [20]:
clamp(10 - median(bm_model).memory / 100, 0, 10)

10.0

## Q5. (30 pts) Starting point

For numerical optimization, a good starting point is critical. Let's start $\boldsymbol{\beta}$ and $\sigma^2$ from the least squares solutions (ignoring intra-individual correlations)
$$
\begin{aligned}
\boldsymbol{\beta}^{(0)} &= \left(\sum_i \mathbf{X}_i^T \mathbf{X}_i\right)^{-1} \left(\sum_i \mathbf{X}_i^T \mathbf{y}_i\right) \\
\sigma^{2(0)} &= \frac{\sum_i \|\mathbf{r}_i^{(0)}\|_2^2}{\sum_i n_i} = \frac{\sum_i \|\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}^{(0)}\|_2^2}{\sum_i n_i}.
\end{aligned}
$$
To get a reasonable starting point for $\boldsymbol{\Sigma} = \mathbf{L} \mathbf{L}^T$, we can minimize the least squares criterion (ignoring the noise variance component)
$$
    \text{minimize} \sum_i \|\mathbf{r}_i^{(0)} \mathbf{r}_i^{(0)T} - \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T \|_{\text{F}}^2.
$$
Derive the minimizer $\boldsymbol{\Sigma}^{(0)}$ (10 pts). 

We implement this start point strategy in the function `init_ls()`.

**Solution**

We can obtain $\boldsymbol{\beta}^{(0)}$ by solving the following linear system in the form of $\mathbf{Ax}=\mathbf{b}$:
$$
\begin{aligned}
 \left(\sum_i \mathbf{X}_i^T \mathbf{X}_i\right)\boldsymbol{\beta}^{(0)} = \sum_i \mathbf{X}_i^T \mathbf{y}_i.
\end{aligned}
$$
For $\sigma^{2(0)}$, we calculate
$$
\begin{aligned}
\sigma^{2(0)} = \frac{\sum_i\left[\mathbf{y}_i^T\mathbf{y}_i + \boldsymbol{\beta}^{(0)T}(\mathbf{X}_i^T\mathbf{X}_i\boldsymbol{\beta}^{(0)} - 2\mathbf{X}_i^T\mathbf{y}_i)\right]}{\sum_i n_i}.
\end{aligned}
$$

To derive $\boldsymbol{\Sigma}^{(0)}$, we need the gradient and hessian. Using the following formula 
$$
\begin{aligned}
\frac{\partial\|\mathbf{C} - \mathbf{AXB}\|_\text{F}^2}{\partial\mathbf{X}}
&= \frac{\partial}{\partial\mathbf{X}}\text{tr}\left((\mathbf{C} - \mathbf{A\Sigma B})(\mathbf{C} - \mathbf{AXB})^T\right)\\
&= \frac{\partial}{\partial\mathbf{X}}\text{tr}(\mathbf{C}\mathbf{C}^T - \mathbf{C}\mathbf{B}^T\mathbf{X}^T\mathbf{A}^T - \mathbf{AXB}\mathbf{C}^T + \mathbf{AXB}\mathbf{B}^T\mathbf{X}^T\mathbf{A}^T)\\
&= \mathbf{O} - \mathbf{A}^T\mathbf{C}\mathbf{B}^T - \mathbf{A}^T\mathbf{C}\mathbf{B}^T + (\mathbf{A}^T\mathbf{AXB}\mathbf{B}^T + \mathbf{A}^T\mathbf{AXB}\mathbf{B}^T)\\
&= -2\mathbf{A}^T(\mathbf{C} - \mathbf{AXB}) \mathbf{B}^T
\end{aligned}
$$
yields our gradient
$$
\begin{aligned}
\frac{\partial\sum_i\|\mathbf{r}_i^{(0)} \mathbf{r}_i^{(0)T} - \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T \|_{\text{F}}^2}{\partial\mathbf{\Sigma}}
= -2\sum_i\left[\mathbf{Z}_i^T(\mathbf{r}_i^{(0)} \mathbf{r}_i^{(0)T} - \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T ) \mathbf{Z}_i\right]
\end{aligned}
$$
and then the hessian
$$
\begin{aligned}
\frac{\partial^2\sum_i\|\mathbf{r}_i^{(0)} \mathbf{r}_i^{(0)T} - \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T \|_{\text{F}}^2}{\partial\mathbf{\Sigma}^2}
= 2\frac{\partial\sum_i\mathbf{Z}_i^T\mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T\mathbf{Z}_i}{\partial\mathbf{\Sigma}}
\end{aligned}
$$
should be psd. Therefore, taking the derivative w.r.t $\mathbf{\Sigma}$ equal to zero gives the minimizer, such that
$$
\sum_i\mathbf{Z}_i^T\mathbf{Z}_i\boldsymbol{\Sigma}^{(0)}\mathbf{Z}_i^T\mathbf{Z}_i = \sum_i(\mathbf{Z}_i^T\mathbf{r}_i^{(0)})(\mathbf{Z}_i^T\mathbf{r}_i^{(0)})^T.
$$
To solve this, we need its (column-major) vectorization. Since $\text{vec}(\mathbf{ABC}) = (\mathbf{C}^T\otimes\mathbf{A})\text{vec}(\mathbf{B})$, we have
$$
\begin{aligned}
\text{vec}\left(\sum_i\mathbf{Z}_i^T\mathbf{Z}_i\boldsymbol{\Sigma}^{(0)}\mathbf{Z}_i^T\mathbf{Z}_i\right)
&= \text{vec}\left(\sum_i(\mathbf{Z}_i^T\mathbf{r}_i^{(0)})(\mathbf{Z}_i^T\mathbf{r}_i^{(0)})^T\right)\\
\Longrightarrow\underbrace{\left(\sum_i\mathbf{Z}_i^T\mathbf{Z}_i\otimes\mathbf{Z}_i^T\mathbf{Z}_i\right)}_{q^2\times q^2}
\underbrace{\text{vec}(\boldsymbol{\Sigma}^{(0)})}_{q^2\times 1}
&= \underbrace{\left(\sum_i \mathbf{Z}_i^T\mathbf{r}_i^{(0)}\otimes\mathbf{Z}_i^T\mathbf{r}_i^{(0)}\right)}_{q^2\times 1}\\
\Longrightarrow\text{vec}(\boldsymbol{\Sigma}^{(0)})
&= \left(\sum_i\mathbf{Z}_i^T\mathbf{Z}_i\otimes\mathbf{Z}_i^T\mathbf{Z}_i\right)^{-1}\left(\sum_i \mathbf{Z}_i^T\mathbf{r}_i^{(0)}\otimes\mathbf{Z}_i^T\mathbf{r}_i^{(0)}\right)
\end{aligned}
$$
with $\boldsymbol{\Sigma}^{(0)} = \mathbf{L}^{(0)}\mathbf{L}^{(0)T}$ by the Cholesky decomposition.

In coding, we can estimate $\sigma^{2(0)}$ and $\boldsymbol{\Sigma}^{(0)}$ simultaneously once $\boldsymbol{\beta}^{(0)}$ is obtained.

In [21]:
"""
    init_ls!(m::LmmModel)

Initialize parameters of a `LmmModel` object from the least squares estimate. 
`m.β`, `m.L`, and `m.σ²` are overwritten with the least squares estimates.
"""
function init_ls!(m::LmmModel{T}) where T <: AbstractFloat
    p, q = size(m.data[1].X, 2), size(m.data[1].Z, 2)
    # TODO: fill m.β, m.L, m.σ² by LS estimates
    # m.β ------------------------------------------------------------------- #
    fill!(m.xtx, 0)
    fill!(m.xty, 0)
    @inbounds for i in 1:length(m.data)
        obs = m.data[i]
        m.xtx .+= obs.xtx # ΣX'X
        m.xty .+= obs.xty # ΣX'y
    end
    LAPACK.posv!('L', m.xtx, m.xty) # solve (ΣX'X)β = ΣX'y
    copy!(m.β, m.xty) # copy solution to m.β

    # m.σ² and m.L ---------------------------------------------------------- #
    fill!(m.σ², 0)
    fill!(m.ztz2, 0)
    fill!(m.ztr2, 0)
    # fill!(m.xty, 0) # cannot reuse m.xty since m.β is overwritten
    @inbounds for i in 1:length(m.data)
        obs = m.data[i]
        ## m.σ² ------------------------------------------------------------ ##
        BLAS.symv!('U', T(1), obs.xtx, m.β, T(-2), copy!(obs.storage_p, obs.xty)) # X'Xβ⁽⁰⁾ - 2X'y
        m.σ² .+= obs.yty .+ dot(m.β, obs.storage_p) # sum(r'r)
        ## m.L ------------------------------------------------------------- ##
        BLAS.kron!(m.storage_2q2q, obs.ztz, obs.ztz) # Z'Z⊗Z'Z
        m.ztz2 .+= m.storage_2q2q # sum(Z'Z⊗Z'Z)
        BLAS.gemv!('N', T(-1), obs.ztx, m.β, T(1), copy!(obs.storage_q, obs.zty)) # Z'r
        BLAS,kron!(m.storage_2q, obs.storage_q, obs.storage_q) # Z'r⊗Z'r
        m.ztr2 .+= m.storage_2q # sum(Z'r⊗Z'r)
    end
    m.σ² ./= sum(length(obs.y) for obs in m.data) # sum(r'r) / sum(n)
    LAPACK.posv!('L', m.ztz2, m.ztr2) # solve sum(Z'Z⊗Z'Z)vec(Σ) = sum(Z'r⊗Z'r)
    # m.L .= reshape(m.ztr2, q, q) # this takes 2 allocations
    @inbounds for i in 1:q, j in 1:q
        m.L[i, j] = m.ztr2[(j-1) * q + i] # copy solution to L (but still Σ)
    end
    LAPACK.potrf!('L', m.L) # Cholesky factor L
end

init_ls!

In [22]:
init_ls!(lmm)
@show logl!(lmm)
@show lmm.β
@show lmm.σ²
@show lmm.L;

logl!(lmm) = -3.356237077697068e6
lmm.β = [0.18207934611476326, 6.50048070099372, -3.4979107842091586, 1.001113296229795, 5.0002519857919285]
lmm.σ² = [5.709004733413663]
lmm.L = [1.4069222734993236 0.07258461003916689 0.05717147035274019; 0.05159105901325529 1.131979211870369 -0.07707942768978568; 0.0406358413891211 -0.06994463586493146 0.9718256360134827]


### Correctness

Your start points should have a log-likelihood larger than -3.3627e6 (10 pts). The points you get are

In [23]:
# this is the points you get
(logl!(lmm) >  -3.3627e6) * 10

10

### Efficiency

The start point should be computed quickly. Otherwise there is no point using it as a starting point. My median run time is 175μs. You get full credit (10 pts) if the median run time is within 1ms.

In [24]:
bm_init = @benchmark init_ls!($lmm)

BenchmarkTools.Trial: 9721 samples with 1 evaluation.
 Range (min … max):  509.125 μs … 820.917 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     512.166 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   512.894 μs ±   6.156 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▃▇█▅▃██▆▄▄▃▃▂▂▁▁▁ ▁                                          ▂
  ▅███████████████████████▇█▇▇▆▆▆▇▆▆▆▅▅▅▆▅▆▆▅▄▅▆▄▃▅▅▄▄▅▃▄▅▄▄▃▅▃ █
  509 μs        Histogram: log(frequency) by time        536 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [25]:
# this is the points you get
clamp(1 / (median(bm_init).time / 1e6) * 10, 0, 10)

10.0

## Q6. NLP via MathOptInterface.jl

We define the NLP problem using the modelling tool [MathOptInterface.jl](https://github.com/jump-dev/MathOptInterface.jl). Start-up code is given below. Modify if necessary to accomodate your own code.

In [26]:
"""
    fit!(m::LmmModel, solver=Ipopt.Optimizer())

Fit an `LmmModel` object by MLE using a nonlinear programming solver. Start point 
should be provided in `m.β`, `m.σ²`, `m.L`.
"""
function fit!(
        m :: LmmModel{T},
        solver = Ipopt.Optimizer()
    ) where T <: AbstractFloat
    p    = size(m.data[1].X, 2)
    q    = size(m.data[1].Z, 2)
    npar = p + ((q * (q + 1)) >> 1) + 1
    # prep the MOI
    MOI.empty!(solver)
    # set lower bounds and upper bounds of parameters
    # q diagonal entries of Cholesky factor L should be >= 0
    # σ² should be >= 0
    lb   = fill(0.0, q + 1)
    ub   = fill(Inf, q + 1)
    NLPBlock = MOI.NLPBlockData(MOI.NLPBoundsPair.(lb, ub), m, true)
    MOI.set(solver, MOI.NLPBlock(), NLPBlock)
    # start point
    params = MOI.add_variables(solver, npar)    
    par0   = Vector{T}(undef, npar)
    modelpar_to_optimpar!(par0, m)    
    for i in 1:npar
        MOI.set(solver, MOI.VariablePrimalStart(), params[i], par0[i])
    end
    MOI.set(solver, MOI.ObjectiveSense(), MOI.MAX_SENSE)
    # optimize
    MOI.optimize!(solver)
    optstat = MOI.get(solver, MOI.TerminationStatus())
    optstat in (MOI.LOCALLY_SOLVED, MOI.ALMOST_LOCALLY_SOLVED) || 
        @warn("Optimization unsuccesful; got $optstat")
    # update parameters and refresh gradient
    xsol = [MOI.get(solver, MOI.VariablePrimal(), MOI.VariableIndex(i)) for i in 1:npar]
    optimpar_to_modelpar!(m, xsol)
    logl!(m, true)
    m
end

"""
    ◺(n::Integer)

Triangular number `n * (n + 1) / 2`.
"""
@inline ◺(n::Integer) = (n * (n + 1)) >> 1

"""
    modelpar_to_optimpar!(par, m)

Translate model parameters in `m` to optimization variables in `par`.
"""
function modelpar_to_optimpar!(
        par :: Vector,
        m   :: LmmModel
    )
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    # β
    copyto!(par, m.β)
    # L
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        par[offset] = m.L[i, j]
        offset += 1
    end
    # σ²
    par[end] = m.σ²[1]
    par
end

"""
    optimpar_to_modelpar!(m, par)

Translate optimization variables in `par` to the model parameters in `m`.
"""
function optimpar_to_modelpar!(
        m   :: LmmModel, 
        par :: Vector
    )
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    # β
    copyto!(m.β, 1, par, 1, p)
    # L
    fill!(m.L, 0)
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        m.L[i, j] = par[offset]
        offset   += 1
    end
    # σ²
    m.σ²[1] = par[end]    
    m
end

function MOI.initialize(
        m                  :: LmmModel, 
        requested_features :: Vector{Symbol}
    )
    for feat in requested_features
        if !(feat in MOI.features_available(m))
            error("Unsupported feature $feat")
        end
    end
end

MOI.features_available(m::LmmModel) = [:Grad, :Hess, :Jac]

function MOI.eval_objective(
        m   :: LmmModel, 
        par :: Vector
    )
    optimpar_to_modelpar!(m, par)
    logl!(m, false) # don't need gradient here
end

function MOI.eval_objective_gradient(
        m    :: LmmModel, 
        grad :: Vector, 
        par  :: Vector
    )
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    optimpar_to_modelpar!(m, par) 
    obj = logl!(m, true)
    # gradient wrt β
    copyto!(grad, m.∇β)
    # gradient wrt L
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        grad[offset] = m.∇L[i, j]
        offset += 1
    end
    # gradient with respect to σ²
    grad[end] = m.∇σ²[1]
    # return objective
    obj
end

function MOI.eval_constraint(m::LmmModel, g, par)
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    gidx   = 1
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        if i == j
            g[gidx] = par[offset]
            gidx   += 1
        end
        offset += 1
    end
    g[end] = par[end]
    g
end

function MOI.jacobian_structure(m::LmmModel)
    p    = size(m.data[1].X, 2)
    q    = size(m.data[1].Z, 2)
    row  = collect(1:(q + 1))
    col  = Int[]
    offset = p + 1
    for j in 1:q, i in j:q
        (i == j) && push!(col, offset)
        offset += 1
    end
    push!(col, offset)
    [(row[i], col[i]) for i in 1:length(row)]
end

MOI.eval_constraint_jacobian(m::LmmModel, J, par) = fill!(J, 1)

function MOI.hessian_lagrangian_structure(m::LmmModel)
    p    = size(m.data[1].X, 2)
    q    = size(m.data[1].Z, 2)    
    q◺   = ◺(q)
    # we work on the upper triangular part of the Hessian
    arr1 = Vector{Int}(undef, ◺(p) + ◺(q◺) + q◺ + 1)
    arr2 = Vector{Int}(undef, ◺(p) + ◺(q◺) + q◺ + 1)
    # Hββ block
    idx  = 1    
    for j in 1:p, i in 1:j
        arr1[idx] = i
        arr2[idx] = j
        idx      += 1
    end
    # HLL block
    for j in 1:q◺, i in 1:j
        arr1[idx] = p + i
        arr2[idx] = p + j
        idx      += 1
    end
    # HLσ² block
    for i in (p + 1):(p + q◺)
        arr1[idx] = i
        arr2[idx] = p + q◺ + 1
        idx      += 1
    end
    # Hσ²σ² block
    arr1[idx] = p + q◺ + 1
    arr2[idx] = p + q◺ + 1
    [(arr1[i], arr2[i]) for i in 1:length(arr1)]
end

function MOI.eval_hessian_lagrangian(
        m   :: LmmModel, 
        H   :: AbstractVector{T},
        par :: AbstractVector{T}, 
        σ   :: T, 
        μ   :: AbstractVector{T}
    ) where {T}    
    p  = size(m.data[1].X, 2)
    q  = size(m.data[1].Z, 2)    
    q◺ = ◺(q)
    optimpar_to_modelpar!(m, par)
    logl!(m, true, true)
    # Hββ block
    idx = 1    
    @inbounds for j in 1:p, i in 1:j
        H[idx] = m.Hββ[i, j]
        idx   += 1
    end
    # HLL block
    @inbounds for j in 1:q◺, i in 1:j
        H[idx] = m.HLL[i, j]
        idx   += 1
    end
    # HLσ² block
    @inbounds for j in 1:q, i in j:q
        H[idx] = m.Hσ²L[i, j]
        idx   += 1
    end
    # Hσ²σ² block
    H[idx] = m.Hσ²σ²[1, 1]
    lmul!(σ, H)
end

## Q7. (20 pts) Test drive

Now we can run any NLP solver (supported by MathOptInterface.jl) to compute the MLE. For grading purpose, let's use the `:LD_MMA` ([Method of Moving Asymptotes](https://nlopt.readthedocs.io/en/latest/NLopt_Algorithms/#mma-method-of-moving-asymptotes-and-ccsa)) algorithm in NLopt.jl.

In [27]:
# initialize from least squares
init_ls!(lmm)
println("objective value at starting point: ", logl!(lmm)); println()

# NLopt (LD_MMA) obj. val = -2.8400587866501966e6
NLopt_solver = NLopt.Optimizer()
MOI.set(NLopt_solver, MOI.RawOptimizerAttribute("algorithm"), :LD_MMA)
@time fit!(lmm, NLopt_solver)

println("objective value at solution: $(logl!(lmm)))")
println("solution values:")
@show lmm.β
@show lmm.σ²
@show lmm.L * transpose(lmm.L)
println("gradient @ solution:")
@show lmm.∇β
@show lmm.∇σ²
@show lmm.∇L
@show norm([lmm.∇β; vec(LowerTriangular(lmm.∇L)); lmm.∇σ²])

objective value at starting point: -3.356237077697068e6

  0.967089 seconds (1.10 M allocations: 72.903 MiB, 85.16% compilation time)
objective value at solution: -2.840058786650191e6)
solution values:
lmm.β = [0.1814776101361502, 6.50038355413275, -3.4998642895680585, 0.9997119258286763, 4.9992294818701675]
lmm.σ² = [1.4987345756953059]
lmm.L * transpose(lmm.L) = [1.9836266207662998 0.06578009077946281 0.055171453028808964; 0.06578009077946281 1.28144118940443 -0.09059652736484758; 0.055171453028808964 -0.09059652736484758 0.943427729674199]
gradient @ solution:
lmm.∇β = [0.020715013384039804, -0.007735285610219078, -0.007759917004352523, -0.0006757573451210419, -0.0007069367911913815]
lmm.∇σ² = [-0.00921637415905252]
lmm.∇L = [-0.00016051920270561057 -0.002991258888290063 0.053537668111773296; 0.0019772269599108577 0.00021146130675053776 0.0013620245882379686; 0.07846763773772997 0.0006943460169284016 0.010634576249044514]
norm([lmm.∇β; vec(LowerTriangular(lmm.∇L)); lmm.∇σ²]) = 0.083

0.08312512919941961

### Correctness

You get 10 points if the following code does not throw `AssertError`.

In [28]:
# objective at solution should be close enough to the optimal
@assert logl!(lmm) > -2.840059e6
# gradient at solution should be small enough
@assert norm([lmm.∇β; vec(LowerTriangular(lmm.∇L)); lmm.∇σ²]) < 0.1

### Efficiency

My median run time is 50ms. You get 10 points if your median time is within 1s(=1000ms).

In [29]:
NLopt_solver = NLopt.Optimizer()
MOI.set(NLopt_solver, MOI.RawOptimizerAttribute("algorithm"), :LD_MMA)
bm_mma = @benchmark fit!($lmm, $(NLopt_solver)) setup=(init_ls!(lmm))

BenchmarkTools.Trial: 36 samples with 1 evaluation.
 Range (min … max):  137.613 ms … 192.441 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     137.780 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   140.538 ms ±  10.622 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                                              
  █▄▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂ ▁
  138 ms           Histogram: frequency by time          192 ms <

 Memory estimate: 18.16 KiB, allocs estimate: 523.

In [30]:
# this is the points you get
clamp(1 / (median(bm_mma).time / 1e9) * 10, 0, 10)

10.0

## Q8. (10 pts) Gradient free vs gradient-based methods

Advantage of using a modelling tool such as MathOptInterface.jl is that we can easily switch the backend solvers. For a research problem, we never know beforehand which solver works best. 

Try different solvers in the NLopt.jl and Ipopt.jl packages. Compare the results in terms of run times (the shorter the better), objective values at solution (the larger the better), and gradients at solution (closer to 0 the better). Summarize what you find.

See this [page](https://nlopt.readthedocs.io/en/latest/NLopt_Algorithms/) for the descriptions of algorithms in NLopt.

Documentation for the Ipopt can be found [here](https://coin-or.github.io/Ipopt/).  

In [31]:
# vector of solvers to compare
solvers = ["NLopt (LN_COBYLA, gradient free)", "NLopt (LD_MMA, gradient-based)", 
    "Ipopt (L-BFGS)"]

function setup_solver(s::String)
    if s == "NLopt (LN_COBYLA, gradient free)"
        solver = NLopt.Optimizer()
        MOI.set(solver, MOI.RawOptimizerAttribute("algorithm"), :LN_COBYLA)
    elseif s == "NLopt (LD_MMA, gradient-based)"
        solver = NLopt.Optimizer()
        MOI.set(solver, MOI.RawOptimizerAttribute("algorithm"), :LD_MMA)
    elseif s == "Ipopt (L-BFGS)"
        solver = Ipopt.Optimizer()
        MOI.set(solver, MOI.RawOptimizerAttribute("print_level"), 0)
        MOI.set(solver, MOI.RawOptimizerAttribute("hessian_approximation"), "limited-memory")
        MOI.set(solver, MOI.RawOptimizerAttribute("tol"), 1e-6)
    elseif s == "Ipopt (use FIM)"
        # Ipopt (use Hessian) obj val = -2.8400587866468e6
        solver = Ipopt.Optimizer()
        MOI.set(solver, MOI.RawOptimizerAttribute("print_level"), 0)        
    else
        error("unrecognized solver $s")
    end
    solver
end

# containers for results
runtime = zeros(length(solvers))
objvals = zeros(length(solvers))
gradnrm = zeros(length(solvers))

for i in 1:length(solvers)
    solver = setup_solver(solvers[i])
    bm = @benchmark fit!($lmm, $solver) setup = (init_ls!(lmm))
    runtime[i] = median(bm).time / 1e9
    objvals[i] = logl!(lmm, true)
    gradnrm[i] = norm([lmm.∇β; vec(LowerTriangular(lmm.∇L)); lmm.∇σ²])
end


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



In [32]:
# display results
pretty_table(
    hcat(solvers, runtime, objvals, gradnrm),
    header = ["Solver", "Runtime", "Log-Like", "Gradiant Norm"],
    formatters = (ft_printf("%5.2f", 2), ft_printf("%8.8f", 3:4))
    )

┌──────────────────────────────────┬─────────┬───────────────────┬───────────────┐
│                           Solver │ Runtime │          Log-Like │ Gradiant Norm │
├──────────────────────────────────┼─────────┼───────────────────┼───────────────┤
│ NLopt (LN_COBYLA, gradient free) │    0.54 │ -2840081.23944276 │  952.86875390 │
│   NLopt (LD_MMA, gradient-based) │    0.14 │ -2840058.78665019 │    0.08312513 │
│                   Ipopt (L-BFGS) │    4.03 │ -2840058.78664680 │    0.00113608 │
└──────────────────────────────────┴─────────┴───────────────────┴───────────────┘


**Solution**

Although `L-BFGS` achieves the largest objective value and the smallest gradients, `LD_MMA` appears to be the best choice: It is more than ten times faster than `L-BFGS` while maintaining almost the same objective value and a gradient norm that is sufficiently close to zero. `LN_COBYLA`(gradient-free) is likely the worst option due to its gradient norm being far from zero.

## Q9. (10 pts) Compare with existing art

Let's compare our method with lme4 package in R and MixedModels.jl package in Julia. Both lme4 and MixedModels.jl are developed mainly by Doug Bates. Summarize what you find.

In [33]:
method  = ["My method", "lme4", "MixedModels.jl"]
runtime = zeros(3)  # record the run times
loglike = zeros(3); # record the log-likelihood at MLE

### Your approach

In [34]:
solver = setup_solver("NLopt (LD_MMA, gradient-based)")
bm_257 = @benchmark fit!($lmm, $solver) setup=(init_ls!(lmm))
runtime[1] = (median(bm_257).time) / 1e9
loglike[1] = logl!(lmm)

-2.840058786650191e6

### lme4 

In [35]:
R"""
library(lme4)
library(readr)
library(magrittr)

testdata <- read_csv("lmm_data.csv")
"""

┌ Warning: RCall.jl: 要求されたパッケージ Matrix をロード中です
└ @ RCall ~/.julia/packages/RCall/dDAVd/src/io.jl:172
┌ Warning: RCall.jl: Rows: 1744977 Columns: 8
│ ── Column specification ────────────────────────────────────────────────────────
│ Delimiter: ","
│ dbl (8): ID, Y, X1, X2, X3, X4, Z1, Z2
│ 
│ ℹ Use `spec()` to retrieve the full column specification for this data.
│ ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
└ @ RCall ~/.julia/packages/RCall/dDAVd/src/io.jl:172


RObject{VecSxp}
# A tibble: 1,744,977 × 8
      ID        Y     X1      X2     X3      X4      Z1      Z2
   <dbl>    <dbl>  <dbl>   <dbl>  <dbl>   <dbl>   <dbl>   <dbl>
 1     1   9.52    0.202 -0.463   0.798  0.734   0.685  -0.570 
 2     1  24.4     1.59  -1.95    1.20   1.43    1.64    0.369 
 3     1  -1.99    0.378 -0.0367  1.63  -1.15   -0.818   2.83  
 4     1 -17.4    -1.88   0.375  -0.498 -0.253   1.56    1.68  
 5     1  -0.0704  0.658 -0.165   0.780 -1.23   -0.0288 -1.09  
 6     1  -0.853   0.458 -0.313  -0.512 -0.800  -0.331   1.98  
 7     1  -1.80    0.220  0.328   1.32  -1.01   -0.363  -0.0703
 8     1   5.88    1.30   0.889  -0.854  0.0714 -0.658  -0.0339
 9     1  -9.21   -1.43  -0.522  -0.119 -0.580  -0.155  -1.89  
10     1 -11.3    -0.468 -0.700   0.872 -1.82    1.80    0.492 
# ℹ 1,744,967 more rows


In [36]:
R"""
rtime <- system.time(mmod <- 
  lmer(Y ~ X1 + X2 + X3 + X4 + (1 + Z1 + Z2 | ID), testdata, REML = FALSE))
"""

┌ Warning: RCall.jl: checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  で警告がありました:
│   Model failed to converge with max|grad| = 0.00245559 (tol = 0.002, component 1)
└ @ RCall ~/.julia/packages/RCall/dDAVd/src/io.jl:172


RObject{RealSxp}
  ユーザ システム     経過 
 107.062    8.083  116.033 


In [37]:
R"""
rtime <- rtime["elapsed"]
summary(mmod)
rlogl <- logLik(mmod)
"""
runtime[2] = @rget rtime
loglike[2] = @rget rlogl;

### MixedModels.jl


In [38]:
testdata = CSV.File("lmm_data.csv", types = Dict(1=>String)) |> DataFrame

Row,ID,Y,X1,X2,X3,X4,Z1,Z2
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,9.52102,0.201821,-0.463234,0.797731,0.73357,0.685476,-0.569622
2,1,24.4063,1.58557,-1.94608,1.19787,1.43149,1.63962,0.369053
3,1,-1.99215,0.378332,-0.0367002,1.63072,-1.15031,-0.817843,2.83422
4,1,-17.4233,-1.8826,0.374561,-0.49786,-0.253248,1.56433,1.67857
5,1,-0.0704245,0.658283,-0.165487,0.77951,-1.22763,-0.0287779,-1.09172
6,1,-0.853357,0.457784,-0.313387,-0.512299,-0.800278,-0.330632,1.97609
7,1,-1.80061,0.220461,0.327879,1.32209,-1.01336,-0.362947,-0.0703055
8,1,5.88119,1.30135,0.88884,-0.853941,0.0714372,-0.658202,-0.0338648
9,1,-9.20504,-1.43248,-0.521638,-0.119287,-0.579596,-0.154869,-1.88707


In [39]:
mj = fit(MixedModel, @formula(Y ~ X1 + X2 + X3 + X4 + (1 + Z1 + Z2 | ID)), testdata)
bm_mm = @benchmark fit(MixedModel, @formula(Y ~ X1 + X2 + X3 + X4 + (1 + Z1 + Z2 | ID)), $testdata)
loglike[3] = loglikelihood(mj)
runtime[3] = median(bm_mm).time / 1e9

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter ~/.julia/packages/ProgressMeter/dMfiC/src/ProgressMeter.jl:594
Minimizing 192    Time: 0:00:00 ( 1.99 ms/it)


0.756524625

In [40]:
display(bm_mm)
mj

BenchmarkTools.Trial: 7 samples with 1 evaluation.
 Range (min … max):  686.260 ms …    1.274 s  ┊ GC (min … max): 1.88% … 24.70%
 Time  (median):     756.525 ms               ┊ GC (median):    4.19%
 Time  (mean ± σ):   855.248 ms ± 212.425 ms  ┊ GC (mean ± σ):  8.46% ±  8.00%

  █ █ █  █      █                 █                           █  
  █▁█▁█▁▁█▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  686 ms           Histogram: frequency by time          1.27 s <

 Memory estimate: 826.57 MiB, allocs estimate: 7850.

|             |    Est. |     SE |        z |      p |   σ_ID |
|:----------- | -------:| ------:| --------:| ------:| ------:|
| (Intercept) |  0.1815 | 0.0444 |     4.08 | <1e-04 | 1.4079 |
| X1          |  6.5004 | 0.0009 |  7000.80 | <1e-99 |        |
| X2          | -3.4999 | 0.0009 | -3768.92 | <1e-99 |        |
| X3          |  0.9997 | 0.0009 |  1078.16 | <1e-99 |        |
| X4          |  4.9992 | 0.0009 |  5391.08 | <1e-99 |        |
| Z2          |         |        |          |        | 0.9712 |
| Z1          |         |        |          |        | 1.1319 |
| Residual    |  1.2242 |        |          |        |        |


### Summary

In [41]:
pretty_table(
    hcat(method, runtime, loglike),
    header = ["Method", "Runtime", "Log-Like"],
    formatters = (ft_printf("%5.2f", 2), ft_printf("%8.6f", 3))
    )

┌────────────────┬─────────┬─────────────────┐
│         Method │ Runtime │        Log-Like │
├────────────────┼─────────┼─────────────────┤
│      My method │    0.14 │ -2840058.786650 │
│           lme4 │  116.03 │ -2840058.786652 │
│ MixedModels.jl │    0.76 │ -2840058.786933 │
└────────────────┴─────────┴─────────────────┘


**Solution**

My method achieved a runtime that is 100 times faster than `lme4` and 5-10 times faster than `MixedModels.jl`, while maintaining the same objective value as these existing methods.

## Q10. Be proud of yourself

Go to your resume/cv and claim you have experience performing analysis on complex longitudinal data sets with millions of records. And you beat current software by XXX fold.